<a href="https://colab.research.google.com/github/AdamMashaka/ML/blob/main/Real_mcv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Nov  8 08:24:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:

%%capture
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install jiwer

In [6]:

from huggingface_hub import notebook_login

notebook_login()

In [7]:

%%capture
!apt install git-lfs

In [8]:

from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")

Downloading:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/869M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset timit_asr downloaded and prepared to /root/.cache/huggingface/datasets/timit_asr/clean/2.0.1/b11b576ddcccbcefa7c9f0c4e6c2a43756f3033adffe0fb686aa61043d0450ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:

timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

In [10]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

In [11]:

from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(timit["train"].remove_columns(["audio", "file"]), num_examples=10)


,text
0,She had your dark suit in greasy wash water all year.
1,Cory attacked the project with extra determination.
2,"The groundhog clearly saw his shadow, but stayed out only a moment."
3,Don't ask me to carry an oily rag like that.
4,The barracuda recoiled from the serpent's poisonous fangs.
5,Don't ask me to carry an oily rag like that.
6,"Crooked, overlapping, twisted, or widely spaced teeth."
7,A domestic automatic washer that will give equivalent results may be used.
8,A toothpaste tube should be squeezed from the bottom.
9,They are both trend-following methods.


In [13]:

import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [14]:
timit = timit.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [15]:
show_random_elements(timit["train"].remove_columns(["audio", "file"]))


,text
0,she had your dark suit in greasy wash water all year
1,count the number of teaspoons of soysauce that you add
2,shipbuilding is a most fascinating process
3,his scalp was blistered from today's hot sun
4,this leaves the ordering of entries variable
5,the drunkard is a social outcast
6,they'll roll off in another day
7,a connoisseur will enjoy this shellfish dish
8,addition and subtraction are learned skills
9,she had your dark suit in greasy wash water all year


In [16]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [17]:
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [19]:

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'t': 0,
 'i': 1,
 'y': 2,
 'b': 3,
 'j': 4,
 'a': 5,
 'm': 6,
 "'": 7,
 'u': 8,
 'r': 9,
 'p': 10,
 'h': 11,
 'e': 12,
 'v': 13,
 'f': 14,
 'c': 15,
 ' ': 16,
 'z': 17,
 'q': 18,
 'w': 19,
 's': 20,
 'x': 21,
 'k': 22,
 'n': 23,
 'g': 24,
 'd': 25,
 'l': 26,
 'o': 27}

In [20]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [21]:

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [22]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [23]:
from transformers import Wav2Vec2CTCTokenizer

In [24]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [56]:
repo_name = "Adam Mashaka Katani"

In [62]:

tokenizer.push_to_hub (Adam Mashaka Katani )

SyntaxError: ignored